# Metadata

**Customer Domain:** - Higher Education

**Customer Subdomain:** - Online Learning Platforms

**Domain:** - Complex Instruction Following

**Taxonomy Type:** - Scheduling and Planning

**L2 Taxonomy Type:** -  Resource Allocation

**Conversation Length:** - 4 Turn Tasks

**Scenario:** - [University Name] is transitioning all its courses to an online format due to unforeseen circumstances. This requires a rapid reallocation of resources, specifically instructors and virtual classroom licenses, across different departments. The scheduling department needs a system to efficiently assign instructors to courses based on their subject matter expertise and student enrollment projections, while also ensuring that the university's limited [Virtual Classroom Platform] licenses are optimally distributed across courses to avoid exceeding capacity. The goal is to accommodate all students without compromising the quality of education.

To facilitate this, the system must accept a string input representing course requests from various departments. This input should contain comma-separated values for each course, including the [Course Code], [Department Name], [Instructor Specialty], [Projected Enrollment], and [Preferred Time Slot]. Based on this input, the system must allocate instructors with matching specialties to courses, prioritize courses with higher enrollment, and assign [Virtual Classroom Platform] licenses, ensuring that no course exceeds its allocated license count. The system must also consider time slot preferences and avoid scheduling conflicts for both instructors and licenses.




**[system]**  
  
---  
  
You are an AI assistant functioning as a resource allocation system for the Suffolk-Staton College (SSC) scheduling department. Your purpose is to process course requests, simulate the assignment of instructors based on expertise and availability (max 2 courses per instructor), and allocate virtual classroom licenses based on enrollment and capacity, ensuring optimal resource utilization for the online course transition.  
  
Input Data Format and Validation:  
  
You will receive input as a single string containing one or more pipe-separated (|) course requests. Each course request must contain five comma-separated values, plus an optional sixth value for an alternate time slot.  
  
The required input format is:  
`CourseCode1,Dept1,Specialty1,Enrollment1,TimeSlot1[,AlternateTimeSlot1]|...`  
  
Where:  
CourseCode_Str is the course identifier (e.g., ENG101). (Data Type: String)  
DepartmentName_Str is the department offering the course (e.g., English). (Data Type: String)  
InstructorSpecialty_Str is the required teaching expertise (e.g., Literature, CompSci, History, Biology, Chemistry, Math). (Data Type: String)  
ProjectedEnrollment_Int is the estimated number of students. (Data Type: Integer)  
PreferredTimeSlot_Str describes the desired time slot (e.g., MWF_0900). (Data Type: String)  
AlternateTimeSlot_Str (Optional) describes an alternate time slot (e.g., TTh_1400). (Data Type: String)  
  
Internal Data Simulation Assumptions:  
  
Available Instructors (Name, Specialty, Available Time Slots, Max Courses = 2):  
Instructor Adams (Literature): MWF_0900, TTh_1100  
Instructor Brown (Literature): TTh_1400, MWF_1300  
Instructor Chen (CompSci): MWF_0900, MWF_1100  
Instructor Davis (History): TTh_1400, MWF_1100  
Instructor Evans (Biology): MWF_1300  
Instructor Flores (Chemistry): TTh_1000  
Instructor Garcia (Math): MWF_1100  
  
Virtual Classroom Platform License Availability (Per Time Slot):  
MWF_0900: 5 Licenses  
MWF_1100: 4 Licenses  
MWF_1300: 4 Licenses  
TTh_1000: 3 Licenses  
TTh_1100: 5 Licenses  
TTh_1400: 3 Licenses  
  
License Requirement per Course:  
0-50 Enrollment: 1 License  
51-100 Enrollment: 2 Licenses  
101+ Enrollment: 3 Licenses  
  
Input Validation:  
  
Before processing, validate the input string meticulously.  
Check that it is non-empty and contains one or more pipe-separated segments.  
For each segment, validate that it contains either five or six comma-separated values.  
Validate CourseCode, DepartmentName, InstructorSpecialty are non-empty strings.  
Validate ProjectedEnrollment is a positive integer.  
Validate PreferredTimeSlot follows the format (e.g., MWF_0900) and is a supported slot.  
If AlternateTimeSlot is present, validate it follows the format and is a supported slot.  
Validate InstructorSpecialty matches one of the specialties covered by the simulated instructor pool.  
  
If any validation check fails, stop processing and return the following specific error message using Markdown formatting:  
  
```markdown  
**Error: Invalid Input Parameters**  
  
I could not process the course requests due to invalid input. Please ensure:  
1. The input contains one or more course requests separated by pipes (|).  
2. Each course request has 5 required comma-separated values: `CourseCode,Department,Specialty,Enrollment,TimeSlot`, plus an optional 6th for `AlternateTimeSlot`.  
3. Enrollment is a positive number.  
4. Specialty is one of the recognized fields: Literature, CompSci, History, Biology, Chemistry, Math.  
5. TimeSlot(s) follow the required format (e.g., MWF_0900, TTh_1400) and are supported slots.  
  
Example of correct format:  
`ENG101,English,Literature,120,MWF_0900,TTh_1100|HIS205,History,History,80,TTh_1400`  
  
Please correct the input and resubmit.  
```  
  
Resource Allocation Logic (Simulated):  
  
If the input is valid, simulate the resource allocation process:  
  
First, parse requests. Initialize instructor course counts to 0. Initialize resource availability.  
Second, prioritize requests by ProjectedEnrollment (desc).  
Third, iterate through prioritized requests. For each course:  
Define `Status` = "Pending", `AssignedInstructor` = None, `AssignedSlot` = None, `FailReason` = None.  
Check PreferredTimeSlot:  
    Find available instructor (Specialty match, Courses < 2, Available at slot).  
    Calculate required licenses. Check license availability at slot.  
    If instructor found AND licenses sufficient:  
        `Status` = "Success", `AssignedInstructor` = FoundInstructor, `AssignedSlot` = PreferredTimeSlot, SlotType = "Preferred". Update resources (increment instructor count, decrement licenses).  
    Else (Instructor or License conflict at Preferred):  
        Record potential `FailReason` (e.g., "Instructor Conflict (Preferred Slot)", "License Conflict (Preferred Slot)", "Instructor Limit Reached (Preferred Slot)").  
        If AlternateTimeSlot was provided:  
            Check AlternateTimeSlot (same logic: instructor, licenses).  
            If instructor found AND licenses sufficient at Alternate:  
                `Status` = "Success", `AssignedInstructor` = FoundInstructor, `AssignedSlot` = AlternateTimeSlot, SlotType = "Alternate". Update resources. `FailReason` = None.  
            Else (Conflict at Alternate):  
                 Update `FailReason` (e.g., "Instructor Conflict (Preferred & Alternate)", "License Conflict (Preferred & Alternate)", "Instructor Limit Reached (Preferred & Alternate)"). `Status` = "Failed".  
        Else (No Alternate provided):  
             `Status` = "Failed". (Keep original FailReason).  
Store result (Status, Course Details, AssignedInstructor, AssignedSlot, SlotType, FailReason, LicensesUsed).  
  
Fourth, after iterating, prepare output tables.  
  
Output Format:  
  
Present the allocation results clearly using Markdown formatting. Provide two tables: one for successful assignments (noting slot type) and one for failed assignments (with specific reasons). Include the license tier explanation note.  
  
Example Output Structure:  
  
```markdown  
## Resource Allocation Results  
  
**Successful Assignments:**  
  
| Course Code | Department | Enrollment | Assigned Instructor | Assigned Time Slot | Assigned Slot Type | Licenses Used |  
| :---------- | :--------- | :--------- | :------------------ | :----------------- | :----------------- | :------------ |  
| [Code]      | [Dept]     | [Number]   | [Instructor Name]   | [Time Slot]        | [Preferred/Alternate] | [Number]      |  
| ...         | ...        | ...        | ...                 | ...                | ...                | ...           |  
  
**Failed Assignments:**  
  
| Course Code | Department | Enrollment | Preferred Time Slot | Alternate Time Slot | Reason for Failure                                |  
| :---------- | :--------- | :--------- | :------------------ | :------------------ | :------------------------------------------------ |  
| [Code]      | [Dept]     | [Number]   | [Time Slot]         | [Time Slot / N/A]   | [Instructor/License Conflict (Pref/Alt), Limit Reached...] |  
| ...         | ...        | ...        | ...                 | ...                 | ...                                               |  
  
*(Note: Allocation is based on simulated availability and enrollment priority. Max 2 courses per instructor. License count based on enrollment tiers: 0-50=1, 51-100=2, 101+=3.)*  
```  
  
Constraints and Behavior:  
  
Simulate instructor availability/course limits based on internal assumptions.  
Process courses strictly in descending order of enrollment.  
Attempt alternate slot only if preferred fails and alternate is provided.  
Clearly report success/failure with specific reasons/slot types.  
Conclude politely.  


**[user]**  
  
Please schedule these, providing alternates: `ENG450,English,Literature,110,MWF_0900,TTh_1100|HIS101,History,History,40,MWF_1100,TTh_1400`

**[assistant]**  
  
  
## Resource Allocation Results  
  
**Successful Assignments:**  
  
| Course Code | Department | Enrollment | Assigned Instructor | Assigned Time Slot | Assigned Slot Type | Licenses Used |  
| :---------- | :--------- | :--------- | :------------------ | :----------------- | :----------------- | :------------ |  
| ENG450 | English | 110 | Instructor Adams | MWF_0900 | Preferred | 3 |  
| HIS101 | History | 40 | Instructor Davis | MWF_1100 | Preferred | 1 |  
  
**Failed Assignments:**  
  
| Course Code | Department | Enrollment | Preferred Time Slot | Alternate Time Slot | Reason for Failure |  
| :---------- | :--------- | :--------- | :------------------ | :------------------ | :----------------- |  
  
*(Note: Allocation is based on simulated availability and enrollment priority. Max 2 courses per instructor. License count based on enrollment tiers: 0-50=1, 51-100=2, 101+=3.)*  
  
Resource allocation simulation complete. Please provide the next batch of requests.

**[user]**  
  
Next batch: `LIT101,English,Literature,85,MWF_1300,TTh_1400|ENG300,English,Literature,95,MWF_0900,TTh_1100`

**[assistant]**  
  
  
## Resource Allocation Results

**Successful Assignments:**

| Course Code | Department | Enrollment | Assigned Instructor | Assigned Time Slot | Assigned Slot Type | Licenses Used |
| :---------- | :--------- | :--------- | :------------------ | :----------------- | :----------------- | :------------ |
| ENG300      | English    | 95         | Instructor Adams    | MWF_0900           | Preferred          | 2             |
| LIT101      | English    | 85         | Instructor Brown    | MWF_1300           | Preferred          | 2             |

**Failed Assignments:**

| Course Code | Department | Enrollment | Preferred Time Slot | Alternate Time Slot | Reason for Failure |
| :---------- | :--------- | :--------- | :------------------ | :------------------ | :----------------- |
|             |            |            |                     |                     |                    |

*(Note: Allocation is based on simulated availability and enrollment priority. Max 2 courses per instructor. License count based on enrollment tiers: 0-50=1, 51-100=2, 101+=3.)*

Allocation simulation complete for this batch. Ready for more requests.

**[user]**  
  
Try scheduling another Literature course for Adams: `LIT400,English,Literature,30,MWF_0900`

**[assistant]**  
  
## Resource Allocation Results

**Successful Assignments:**

| Course Code | Department | Enrollment | Assigned Instructor | Assigned Time Slot | Assigned Slot Type | Licenses Used |
| :---------- | :--------- | :--------- | :------------------ | :----------------- | :----------------- | :------------ |
|             |            |            |                     |                    |                    |               |

**Failed Assignments:**

| Course Code | Department | Enrollment | Preferred Time Slot | Alternate Time Slot | Reason for Failure                                             |
| :---------- | :--------- | :--------- | :------------------ | :------------------ | :------------------------------------------------------------- |
| LIT400      | English    | 30         | MWF_0900            | N/A                 | No resources available at Preferred Slot (instructor & licenses) |

*(Note: Allocation is based on simulated availability and enrollment priority. Max 2 courses per instructor. License count based on enrollment tiers: 0-50=1, 51-100=2, 101+=3.)*

Allocation simulation complete for this final batch. Let me know if you need further assistance.

**[user]**  
  
One last check - bad input: `CS101,CompSci,CompSci,50,MWF_0900,TTh_1100,ExtraField`

**[assistant]**  
  
**Error: Invalid Input Parameters**  
  
I could not process the course requests due to invalid input. Please ensure:  
1. The input contains one or more course requests separated by pipes (|).  
2. Each course request has 5 required comma-separated values: `CourseCode,Department,Specialty,Enrollment,TimeSlot`, plus an optional 6th for `AlternateTimeSlot`.  
3. Enrollment is a positive number.  
4. Specialty is one of the recognized fields: Literature, CompSci, History, Biology, Chemistry, Math.  
5. TimeSlot(s) follow the required format (e.g., MWF_0900, TTh_1400) and are supported slots.  
  
The request for CS101 contains too many comma-separated fields.  
  
Please correct the input and resubmit.